In [ ]:
%matplotlib inline
import lightkurve as lk
from lightkurve_ext import download_lightcurvefiles
import tic_plot as tplt

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as matplotlib

import pandas as pd
import astropy as astropy
from astropy.io import fits

In [ ]:
# Try to plot that handles discontinuity

x = np.array([0, 0.5, 1, 1.5, 3, 3.5, 4, 4.5])
y = np.array([2, 3, 2, 3, 4, 5, 4, 5])
x_diff = np.diff(x, prepend=-0.4)


In [ ]:
from IPython.core.interactiveshell import InteractiveShell
# show all outputs, not just the last expression
# see https://ipython.readthedocs.io/en/stable/config/options/terminal.html#configtrait-InteractiveShell.ast_node_interactivity
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
x_diff
y
y_masked = np.ma.masked_where(x_diff > 0.5, y)
y_masked

def mask_discontinous(x, y, min_x_diff):
    x_diff = np.diff(x, prepend=-min_x_diff)
    return np.ma.masked_where(x_diff > min_x_diff, y)

plt.plot(x, mask_discontinous(x, y, 0.6))

In [ ]:
# restore default
InteractiveShell.ast_node_interactivity = "last_expr"

In [ ]:
# Possible Candidates, not a TOI yet
# the dips look a tad V-shape though.
tic1 = 232612416

# Load the light_curves into collection
lcf_coll = download_lightcurvefiles(f"TIC{tic1}", mission='TESS', download_dir='./data')
print(len(lcf_coll))


In [ ]:
# lcf_coll.PDCSAP_FLUX.plot()

lcf14 = lcf_coll[0]
lc14 = lcf14.PDCSAP_FLUX
lc14.scatter()

fig = plt.figure(figsize=(12, 6))
fig.gca().scatter(lc14.time, lc14.flux)

lcf_coll

# foo = lk.search.search_lightcurvefile(target='TIC232612416', mission='TESS').download_all()
# foo

In [ ]:
lc14.quality_bitmask
print('unit=_{}_'.format(lc14.normalize().flux_unit))

In [ ]:
time_s = np.nanmin(lc14.time)
lc14.time_orig = lc14.time
lc14.time_rel = lc14.time_orig - time_s
lc14.time = lc14.time_rel
ax = lc14.plot()
ax.set_xlabel('Time, relative [BTJD days]')

# Proposed new method
# LC being a LightCurve
# lc14.plot(use_relative_time=True)

fig = plt.figure(figsize=(8,8))
st=9000
end=10000
fig.gca().scatter(lc14.cadenceno[st:end], lc14.time[st:end])
len(lc14.cadenceno)
# lc14.time_format

lc14.cadenceno[13] - lc14.cadenceno[0]
(lc14.time[13] - lc14.time[0]) * 24 * 60 # in minutes


In [ ]:
lk.__version__

In [ ]:
tplt.plot_all(lcf_coll)

In [ ]:
if False: 
    fig = plt.figure(figsize=(30,15))
    tplt.plot_lcf_flux_delta(tplt.lcf_of_sector(lcf_coll, 15), fig.gca()
                     , xmin=1734.5, xmax=1735.5
                       )


In [ ]:
def plot_selected(lcf_coll):                      
    matplotlib.rcParams.update({'font.size':18}) 
    matplotlib.rcParams.update({'font.family':'sans-serif'})
    ax = tplt.plot_n_annotate_lcf(tplt.lcf_of_sector(lcf_coll, 14), ax=tplt.lcf_fig().gca()
                        , t0=1687.13, t_start=1687.06, t_end=1687.21
#                         , t0=1704.58, t_start=1704.51, t_end=1704.65
                       )
    tplt.plot_n_annotate_lcf(tplt.lcf_of_sector(lcf_coll, 15), ax=tplt.lcf_fig().gca()
                        , t0=1735.10, t_start=1735.03, t_end=1735.17
                       )
#     print(f"Period ~= {as_4decimal(1622.5 - 1538.6)} days")
    return 

plot_selected(lcf_coll)

print("Sectors: " + str(list(map(lambda lcf: lcf.header()['SECTOR'], lcf_coll))))


In [ ]:
# show more properties of one lightcurve

print(list(map(lambda lc: lc.header()['SECTOR'], lcf_coll)))
print(list(map(lambda lc: lc.header()['CAMERA'], lcf_coll)))
print(list(map(lambda lc: lc.header()['DATE-OBS'], lcf_coll)))

lcf14 = tplt.lcf_of_sector(lcf_coll, 14)
lc14 = lcf14.PDCSAP_FLUX.normalize(unit='percent')

lc14.show_properties()

# noise estimates
# transit_duration=90, 90 cadence, roughly 3 hours with in data with 2-min cadence
print (f"CDPP: {lc14.estimate_cdpp(transit_duration=90)}")
print (f"CDPP - flatten: {lc14.flatten().estimate_cdpp(transit_duration=90)}")
print (f"CDPP - remove outlines: {lc14.remove_outliers().estimate_cdpp(transit_duration=90)}")



In [ ]:
def newplot(title):
    fig = plt.figure(figsize=(15, 6))
    fig.gca().set_title(title)
    return fig

fig = newplot('Base plot, no processing')
lc14.scatter(ax=fig.gca())


In [ ]:
# default 5 sigma seems to be okay
fig = newplot('remove_outliers()')
lc14.remove_outliers().scatter(ax=fig.gca())


In [ ]:
fig = newplot('flatten() - too aggressive')
lc14.flatten().scatter(ax=fig.gca())


In [ ]:
corrector = lc14.to_corrector(method='sff') # SFFCorrector
lc_corr = corrector.correct()
fig = newplot('w/ SFFCorrector - not much difference')
lc_corr.scatter(ax=fig.gca())


In [ ]:
# SFFCorrector relies on centroid, show centroid

fig = plt.figure(figsize=(12,12))
fig.gca().scatter(lc14.centroid_col, lc14.centroid_row)
fig.gca().set_title('Centroids')


In [ ]:
lcf15 = tplt.lcf_of_sector(download_lightcurvefiles(f"TIC232526073", mission='TESS', download_dir='./data')
                          , 15)


In [ ]:
tplt.animate_centroids(lcf15
                       , fig=plt.figure(figsize=(8,8))
                       , num_obs_per_frame = 480
#                        , frames=list(map(lambda n: int(n), np.linspace(720, 720 + 720, num=24, endpoint=True)))
                       , interval=750
                       , c='blue', accumulative=False, use_relative_time=True)


In [ ]:
# binsize 15 is approximately half an hour, so it's roughly the same as mooving average of 30minues
lc14.bin(binsize=15).plot(title="Binned, size=15")


In [ ]:
fig = newplot('Folded Lightcurve (1 sector), outliers removed')
# t0, period by eyeballing the lightcurve
t0 = 1687.13
period = 4.3625
lc14.remove_outliers().fold(period, t0=t0).scatter(ax=fig.gca())

fig = newplot('Folded Lightcurve (all sectors), outliers removed')
lcf_coll.PDCSAP_FLUX.stitch().normalize().remove_outliers().fold(period, t0=t0).scatter(ax=fig.gca())


In [ ]:
# Detrending, with moving average as the fit
# - based on https://oldtalk.planethunters.org/science/discussions/DPH100ht7b

df14_ma = tplt.add_flux_moving_average(lc14, '30min')
df14_ma['flux_detrended'] = df14_ma['flux'] - df14_ma['flux_mavg'] + 100.0

# OPEN: I am not quite sure how to interpret / use detrended flux 
if True:
    plt.figure(figsize=(15,10))
    plt.scatter(df14_ma['time'], df14_ma['flux'], label="base")
    plt.scatter(df14_ma['time'], df14_ma['flux_mavg'], label="moving avg")
    plt.legend()
    plt.figure(figsize=(15,10))
    plt.scatter(df14_ma['time'], df14_ma['flux_detrended'], label='detrended')
    plt.legend()



In [ ]:
# show details of one dip
df14 = lcf14.PDCSAP_FLUX.normalize(unit='percent').to_pandas(columns=['time', 'flux', 'centroid_row', 'centroid_col'])
df14[(df14.time >= 1704.51) & (df14.time <= 1704.65)]


In [ ]:
print("Sector Observation time range:")
for i in range(0, len(lcf_coll)):
    lc_cur = lcf_coll[i].get_lightcurve('PDCSAP_FLUX')
    print(f"  Sector {lcf_coll[i].header()['SECTOR']}: {lc_cur.time.min()} - {lc_cur.time.max()}")

# use the interactive transit period detection
#   caveat: un-sure if combining observations over time make sense for the algorithm
lcf_coll.PDCSAP_FLUX.stitch().interact_bls()
